# __IOC Hunting__
### By: Alejandro Perez
----------------------------


## Introduction
This Notebook will be used as an test case where we will use the ThreatFox API
to pull the most recent IOC's that have been verified and then pivot from those
to using Data from other Sources such as VirusTotal, and GreyNoise



In [1]:
from src.ThreatFoxUtils import ThreatFox
from src.VirusTotalUtils import VirusTotal
from src.GreyNoiseUtils import Greynoise
import pandas as pd
from pandas import json_normalize
import ipywidgets as widgets
from IPython.display import clear_output
import json
import time
pd.set_option('display.max_colwidth', 0)

In [2]:
# This will use the Config File in order to 
with open('config.json') as config:
    configData = json.load(config)

In [3]:
# Instantiating the VirusTotal API Key from the config file and saving it in a Variable
VT = VirusTotal(ApiKey=configData['VirusTotal Api Key'])

In [4]:
# Gathering the IOC's for the past day
IocData = ThreatFox.query_ioc_database(number_of_days=1)

[-] Gathering IOC's from past 1 days...
[+] IOC Data has been Gathered


In [5]:
# Displaying 10 random Samples from the Dataset
IocData.sample(n=10)

,id,ioc,threat_type,threat_type_desc,ioc_type,ioc_type_desc,malware,malware_printable,malware_alias,malware_malpedia,confidence_level,first_seen,last_seen,reference,reporter,tags
35,9305,ca831dbe47d3934cd62ff5829cb585d4cb2fdec2b5380f775867f9666448ee82,payload,Indicator that identifies a malware sample (payload),sha256_hash,SHA256 hash of a malware sample (payload),win.agent_tesla,Agent Tesla,"AgenTesla,AgentTesla,Negasteal",https://malpedia.caad.fkie.fraunhofer.de/details/win.agent_tesla,50,2021-04-20 17:03:45 UTC,None,https://twitter.com/RedBeardIOCs/status/1384553357747146753,Virus_Deck,[AgentTesla]
43,9301,806543c5b925cac1e0db9abbb28ae330fbb323d68946599fdc32402bf7e1f49a,payload,Indicator that identifies a malware sample (payload),sha256_hash,SHA256 hash of a malware sample (payload),win.agent_tesla,Agent Tesla,"AgenTesla,AgentTesla,Negasteal",https://malpedia.caad.fkie.fraunhofer.de/details/win.agent_tesla,50,2021-04-20 17:03:44 UTC,None,https://twitter.com/RedBeardIOCs/status/1384553356249681924,Virus_Deck,[AgentTesla]
95,9242,46.183.220.67:5456,botnet_cc,Indicator that identifies a botnet command&control server (C&C),ip:port,ip:port combination that is used for botnet Command&control (C&C),win.nanocore,Nanocore RAT,"Nancrat,NanoCore",https://malpedia.caad.fkie.fraunhofer.de/details/win.nanocore,100,2021-04-20 11:20:24 UTC,None,None,abuse_ch,"[NanoCore, RAT]"
144,9193,http://45.76.199.119/g.pixel,botnet_cc,Indicator that identifies a botnet command&control server (C&C),url,URL that is used for botnet Command&control (C&C),win.cobalt_strike,Cobalt Strike,"Agentemis,BEACON,CobaltStrike",https://malpedia.caad.fkie.fraunhofer.de/details/win.cobalt_strike,100,2021-04-20 06:52:51 UTC,None,https://twitter.com/MichalKoczwara/status/1384193759248752645,MichalKoczwara,[Cobalt Strike]
38,9296,0ee3011629a86e28b5f86355426f843ea98ee8c0220335f99094f5a78af1a453,payload,Indicator that identifies a malware sample (payload),sha256_hash,SHA256 hash of a malware sample (payload),win.agent_tesla,Agent Tesla,"AgenTesla,AgentTesla,Negasteal",https://malpedia.caad.fkie.fraunhofer.de/details/win.agent_tesla,50,2021-04-20 17:03:44 UTC,None,https://twitter.com/RedBeardIOCs/status/1384553356249681924,Virus_Deck,[AgentTesla]
71,9266,45.67.228.147:15980,botnet_cc,Indicator that identifies a botnet command&control server (C&C),ip:port,ip:port combination that is used for botnet Command&control (C&C),win.redline_stealer,RedLine Stealer,None,https://malpedia.caad.fkie.fraunhofer.de/details/win.redline_stealer,100,2021-04-20 13:20:23 UTC,None,None,abuse_ch,[RedLineStealer]
10,9324,5.199.162.115:443,botnet_cc,Indicator that identifies a botnet command&control server (C&C),ip:port,ip:port combination that is used for botnet Command&control (C&C),win.cobalt_strike,Cobalt Strike,"Agentemis,BEACON,CobaltStrike",https://malpedia.caad.fkie.fraunhofer.de/details/win.cobalt_strike,100,2021-04-20 22:18:16 UTC,None,https://twitter.com/MichalKoczwara/,MichalKoczwara,[Cobalt Strike]
119,9219,http://221.230.142.27:443/jquery-3.3.1.min.js,botnet_cc,Indicator that identifies a botnet command&control server (C&C),url,URL that is used for botnet Command&control (C&C),win.cobalt_strike,Cobalt Strike,"Agentemis,BEACON,CobaltStrike",https://malpedia.caad.fkie.fraunhofer.de/details/win.cobalt_strike,75,2021-04-20 08:05:12 UTC,None,https://bazaar.abuse.ch/sample/58c874f4e6d5747159c6bde9037fbe4769279a308e04271a527e0e3a07a964c2/,abuse_ch,[CobaltStrike]
99,9239,f6d897549a0349c59c2a1da1068949d59993b18aa651cce7cc7313816ff4f867,payload,Indicator that identifies a malware sample (payload),sha256_hash,SHA256 hash of a malware sample (payload),win.agent_tesla,Agent Tesla,"AgenTesla,AgentTesla,Negasteal",https://malpedia.caad.fkie.fraunhofer.de/details/win.agent_tesla,50,2021-04-20 10:03:55 UTC,None,https://twitter.com/RedBeardIOCs/status/1384447704043036672,Virus_Deck,[AgentTesla]
58,9278,9406ad52a87d220e0eae7b7a65a1870a72df536649d9600aca18ddce2263001f,payload,Indicator that identifies a 

## __GreyNoise__
------------
In this Section, We will use Greynoise which a Service that will return information on IP Addresses in order to rule out any Whitenoise when it comes to scanning. Sources like Shodan, and Censys scan the Internet regularly but mean no harm when doing so, but this service will assist us in finding the IP Addresses that could be scanning maliciously, if there are any in the current dataset.

In order to do this we will take the Dataset from ThreatFox, Filter for IP Addresses, and then Apply it and have a Dataframe returned to us if there are any malicious IP Addresses to track.

In [6]:
# Filtering the Data Set for solely IP:Port Entries
Suspicious_IP = IocData[IocData['ioc_type']=='ip:port']

# Displaying the newly filtered Dataframe (only first 5 rows)
display(Suspicious_IP.head(5))

,id,ioc,threat_type,threat_type_desc,ioc_type,ioc_type_desc,malware,malware_printable,malware_alias,malware_malpedia,confidence_level,first_seen,last_seen,reference,reporter,tags
0,9337,185.140.53.54:6687,botnet_cc,Indicator that identifies a botnet command&control server (C&C),ip:port,ip:port combination that is used for botnet Command&control (C&C),win.nanocore,Nanocore RAT,"Nancrat,NanoCore",https://malpedia.caad.fkie.fraunhofer.de/details/win.nanocore,100,2021-04-21 02:45:27 UTC,None,None,abuse_ch,"[NanoCore, RAT]"
1,9336,172.111.168.19:6381,botnet_cc,Indicator that identifies a botnet command&control server (C&C),ip:port,ip:port combination that is used for botnet Command&control (C&C),win.asyncrat,AsyncRAT,None,https://malpedia.caad.fkie.fraunhofer.de/details/win.asyncrat,100,2021-04-21 01:55:25 UTC,None,None,abuse_ch,"[asyncrat, RAT]"
2,9335,5.101.66.180:80,botnet_cc,Indicator that identifies a botnet command&control server (C&C),ip:port,ip:port combination that is used for botnet Command&control (C&C),win.redline_stealer,RedLine Stealer,None,https://malpedia.caad.fkie.fraunhofer.de/details/win.redline_stealer,100,2021-04-21 00:55:57 UTC,None,None,abuse_ch,[RedLineStealer]
4,9333,185.162.235.111:80,botnet_cc,Indicator that identifies a botnet command&control server (C&C),ip:port,ip:port combination that is used for botnet Command&control (C&C),win.cobalt_strike,Cobalt Strike,"Agentemis,BEACON,CobaltStrike",https://malpedia.caad.fkie.fraunhofer.de/details/win.cobalt_strike,50,2021-04-20 22:20:04 UTC,None,https://twitter.com/MichalKoczwara/,MichalKoczwara,[Cobalt Strike]
5,9331,174.138.0.82:80,botnet_cc,Indicator that identifies a botnet command&control server (C&C),ip:port,ip:port combination that is used for botnet Command&control (C&C),win.cobalt_strike,Cobalt Strike,"Agentemis,BEACON,CobaltStrike",https://malpedia.caad.fkie.fraunhofer.de/details/win.cobalt_strike,50,2021-04-20 22:20:03 UTC,None,https://twitter.com/MichalKoczwara/,MichalKoczwara,[Cobalt Strike]


# Creating List object to store GreyNoise data
GreyNoise_Data_List  = []

# Iterating through the Suspicious_IP Data
print("[-] Scanning IP's...")
for IP_Port in IocData['ioc']:
    # Splitting the IP:Port Combination for solely the IP Address
    IP = IP_Port.split(":")[0]
    # Using our Custom Function for GreyNoise to return the Information to us
    GreyNoise_Scan = Greynoise.lookup_IP(IP)
    # Checks to catch any errors such as this one and will pass over it
    if GreyNoise_Scan['message'] == "Request is not a valid routable IPv4 address":
        pass
    else:
        # Appending the Data to the list we made above
        GreyNoise_Data_List.append(GreyNoise_Scan)

print("[+] Scan has completed")


In [7]:
# Takes the List of Data and flattens the Json into a Dataframe
GreyNoiseDataframe = json_normalize(GreyNoise_Data_List)
# Displays the Dataframe
GreyNoiseDataframe

NameError: name 'GreyNoise_Data_List' is not defined

## __VirusTotal__
----------------
This Section is Dedicated to grabbing the Hashes from ThreatFox and pull reports from VirusTotal on them. In order to do this, we will follow the same methodology and filter out the hashes from our ThreatFox Dataset and then begin to pull information from VirusTotal.

However this section will be interactive with a small Widget tool to search for reports on the hash

In [8]:
# Filters for Suspicious Hashes
Suspicious_Hashes = IocData[IocData['ioc_type']=='sha256_hash']

In [9]:
# Displays the Suspicious Hashes first 5 rows
Suspicious_Hashes.head(5)

,id,ioc,threat_type,threat_type_desc,ioc_type,ioc_type_desc,malware,malware_printable,malware_alias,malware_malpedia,confidence_level,first_seen,last_seen,reference,reporter,tags
27,9307,c50583bdebf9d38a325a1bb74e769cca7fb9fee7722aedb360b8fec069775f52,payload,Indicator that identifies a malware sample (payload),sha256_hash,SHA256 hash of a malware sample (payload),win.agent_tesla,Agent Tesla,"AgenTesla,AgentTesla,Negasteal",https://malpedia.caad.fkie.fraunhofer.de/details/win.agent_tesla,50,2021-04-20 18:03:50 UTC,None,https://twitter.com/RedBeardIOCs/status/1384568479035142144,Virus_Deck,[AgentTesla]
28,9308,b8ff56a3f38384dcd38e3eee204d84a046364226151687fba057c75fb987c15d,payload,Indicator that identifies a malware sample (payload),sha256_hash,SHA256 hash of a malware sample (payload),win.agent_tesla,Agent Tesla,"AgenTesla,AgentTesla,Negasteal",https://malpedia.caad.fkie.fraunhofer.de/details/win.agent_tesla,50,2021-04-20 18:03:50 UTC,None,https://twitter.com/RedBeardIOCs/status/1384568479035142144,Virus_Deck,[AgentTesla]
29,9309,e20251932fbb15ab76bded960ee7f6a4dc33847603e1a410f213636df14eba58,payload,Indicator that identifies a malware sample (payload),sha256_hash,SHA256 hash of a malware sample (payload),win.agent_tesla,Agent Tesla,"AgenTesla,AgentTesla,Negasteal",https://malpedia.caad.fkie.fraunhofer.de/details/win.agent_tesla,50,2021-04-20 18:03:50 UTC,None,https://twitter.com/RedBeardIOCs/status/1384568479035142144,Virus_Deck,[AgentTesla]
30,9310,4d5efc07157713342c2b04e645ad4df03a689c09f92c928024518ebffa883bfd,payload,Indicator that identifies a malware sample (payload),sha256_hash,SHA256 hash of a malware sample (payload),win.agent_tesla,Agent Tesla,"AgenTesla,AgentTesla,Negasteal",https://malpedia.caad.fkie.fraunhofer.de/details/win.agent_tesla,50,2021-04-20 18:03:50 UTC,None,https://twitter.com/RedBeardIOCs/status/1384568479035142144,Virus_Deck,[AgentTesla]
32,9302,67e1c4b84161a155a5b9a8afcf7e62f357f6f5d12576160d4f0559d86f18173a,payload,Indicator that identifies a malware sample (payload),sha256_hash,SHA256 hash of a malware sample (payload),win.agent_tesla,Agent Tesla,"AgenTesla,AgentTesla,Negasteal",https://malpedia.caad.fkie.fraunhofer.de/details/win.agent_tesla,50,2021-04-20 17:03:45 UTC,None,https://twitter.com/RedBeardIOCs/status/1384553357747146753,Virus_Deck,[AgentTesla]


In [10]:
# Building the Output of the Widgets
VT_File_Report_Output = widgets.Output()

# Building the Hash Selection Dropdown Menu
VT_Hash_Selection = widgets.Dropdown(
    options = Suspicious_Hashes['ioc'].tolist(),
    description = 'Hashes:'
)

# Building the Button for the VT Hash Search
VT_SearchButton = widgets.Button(
    description = 'Search'
)

# adding action to the VT Search Button
def VT_Search_Action(x):
    with VT_File_Report_Output:
        clear_output()
        # Runs the Report Search for the Hash
        Report = VT.get_file_information(FileHash=VT_Hash_Selection.value)
        
        # Checks if what is returned back is a Dataframe,
        # which will return a Transposed Dataframe for easier readability
        if isinstance(Report, pd.DataFrame):
            display(Report.T)
        
        # If it is not a Dataframe, There was an error in finding the
        # Hash and so it will return a String error
        else:
            print(Report)

            
# Attaching the Action Function to the button
VT_SearchButton.on_click(VT_Search_Action)

# Creating a Vertical Box to handle the widgets
widgets.VBox([VT_Hash_Selection, VT_SearchButton, VT_File_Report_Output])